In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab04.ipynb")

<a id='verytop'></a>

# Lab 4: Applications of Eigenvectors and Eigenvalues

## Due Date: Tues, Oct 28th 11:59 PM on Gradescope


### Detailed Submission Instructions Are Provided at the end of this Notebook


## Collaboration Policy

A key step in learning and retention is **creating solutions on your own.**   Below are examples of acceptable vs unacceptable use of resources and collaboration when doing lab assignments in CSCI 2820.


The following would be some **examples of cheating** when working on HW assignments in CSCI 2820.  Any of these constitute a **violation of the course's collaboration policy and will result in an F in the course and a trip to the honor council**.   


 - Consulting web pages that may have a solution to a given lab problem or one similar is cheating.  However, consulting notes from the class videos, and web pages that explain the material taught in class but do NOT show a solution to the lab problem in question are permissible to view.  Clearly, there's a fuzzy line here between a valid use of resources and cheating. To avoid this line, one should merely consult the course videos, the course textbooks, and references that contain syntax and/or formulas.
 - Copying a segment of code or math solution of three lines or more from another student from a printout, handwritten copy, or by looking at their computer screen 
 - Allowing another student to copy a segment of your code or math solution of three lines or more
 - Taking a copy of another student's work (or a solution found online) and then editing that copy
 - Reading someone else’s solution to a problem on the lab before writing your own.
 - Asking someone to write all or part of a program or solution for you.
 - Asking someone else for the code necessary to fix the error for you, other than for simple syntactical errors
 


On the other hand, the following are some **examples of things which would NOT usually be
considered to be cheating**:
 - Working on a lab problem on your own first and then discussing with a classmate a particular part in the problem solution where you are stuck.  After clarifying any questions you should then continue to write your solution independently.
 - Asking someone (or searching online) how a particular construct in the language works.
 - Asking someone (or searching online) how to formulate a particular construct in the language.
 - Asking someone for help in finding an error in your program.  
 - Asking someone why a particular construct does not work as you expected in a given program.
   

To test whether you are truly doing your own work and retaining what you've learned you should be able to easily reproduce from scratch and explain a lab solution that was your own when asked in office hours by a TA/Instructor or on a quiz/exam.   


If you have difficulty in formulating the general solution to a problem on your own, or
you have difficulty in translating that general solution into a program, it is advisable to see
your instructor or teaching assistant rather than another student as this situation can easily
lead to a, possibly inadvertent, cheating situation.

We are here to help!  Visit office Hours and/or post questions on Piazza!



## Grading
Grading is broken down into autograded answers and manually graded answers. 

For autograded answers, the results of your code are compared to provided tests.

For manually graded answers you must show and explain all steps.  Graders will evaluate how well you answered the question and/or fulfilled the requirements of the question.


<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />


In [ ]:
# import useful libraries
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp

sp.init_printing(use_unicode=True)


# Function needed to run in-notebook tests
import hashlib

def get_hash(num):
    """Helper function for assessing correctness"""
    return hashlib.md5(str(num).encode()).hexdigest()


def get_array_hash_normalized(arr):
    """
    Hash numpy array ignoring dtype differences (values only),
    treating -0.0 as 0.0 and canonicalizing NaNs.
    """
    arr = np.ascontiguousarray(arr, dtype=np.float64)

    # View as unsigned integers so we can manipulate bits
    view = arr.view(np.uint64)

    # Mask for the exponent+mantissa (everything except the sign bit)
    mant_exp_mask = (1 << 63) - 1

    # For entries that are exactly zero (pos or neg), clear sign bit
    zero_mask = (view & mant_exp_mask) == 0
    view[zero_mask] = 0

    # Canonicalize NaNs: all NaNs get the same payload
    nan_mask = np.isnan(arr)
    if nan_mask.any():
        arr[nan_mask] = np.float64(np.nan)

    return hashlib.md5(arr.tobytes() + str(arr.shape).encode()).hexdigest()

## Recall: Matrix and Vector Operations in SymPy

For this lab we will return to using Symbolic Python.  This will help ensure we don't have floating point errors when dealing with different matrices.


Recall: to create the matrix
\begin{equation}
\mathbf{B} = \left[\begin{array}{r} 1 &  3 \\  2 & 4 \\ -3 &5 \end{array}\right]
\end{equation}
in SymPy we can use the following syntax:

In [ ]:
B = sp.Matrix([[1,3],[2,4], [-3,5]])
B

Recall: We can find RREF using:


In [ ]:
B.rref()

For an invertible matrix, you can find the inverse in Sympy using the following syntax:  `**-1`


In [ ]:
A = sp.Matrix([[1,3],[2,4]])
A

In [ ]:
A**-1

You can calculate other powers of matricesin SymPy using `**`

In [ ]:
A**2

### Calculating Eigenvalues and Eigenvectors in Sympy

To find the eigenvectors of a matrix, in Sympy we can use `eigenvects()`. This returns a list of tuples of the form (eigenvalue, algebraic_multiplicity, [eigenvectors]).

In [ ]:
A = sp.Matrix([[2,-1],[-1,2]])

In [ ]:
eigen_info= A.eigenvects()
eigen_info

In [ ]:
# To list just the eigenvectors:

eigenvectors = []
for eigenvalue, multiplicity, vecs in eigen_info:
    eigenvectors.extend(vecs)

eigenvectors

# Diagonal factorization

The calculation that we used to verify the eigenvalues and eigenvectors is also very useful to construct another important matrix factorization.  Suppose that $A$ is an $n\times n$ matrix, $D$ is a diagonal matrix with the eigenvalues of $A$ along its diagonal, and $P$ is the $n\times n$ matrix with the eigenvectors of $A$ as its columns.  We have just seen that $AP=PD$.  If $P$ is invertible, we may also write $A=PDP^{-1}$, which is known as the **diagonalization** of $A$.

The diagonalization of $A$ is important because it provides us with a complete description of the action of $A$ in terms of its eigenvectors.  Consider an arbitrary vector $X$, and the product $AX$, computed by using the three factors in the diagonalization.

- $P^{-1}X$ computes the coordinates of $X$ in terms of the eigenvectors of $A$.
- Multiplication by $D$ then simply scales each coordinate by the corresponding eigenvalue.
- Multiplication by $P$ gives the results with respect to the standard basis.

This understanding does not provide a more efficient way of computing the product $AX$, but it does provide a much more general way of understanding the result of a matrix-vector multiplication.  As we will see in the next section, the diagonalization also provides a significant shortcut in computing powers of $A$.  

It should be noted that diagonalization of an $n\times n$ matrix $A$ is not possible when the eigenvectors of $A$ form a linearly dependent set.  In that case the eigenvectors do not span $\mathbb{R}^n$, which means that not every $X$ in $\mathbb{R}^n$ can be written as a linear combination of the eigenvectors.  In terms of the computation above, $P$ will not be invertible exactly in this case.

### Diagonalization in SymPy

We can use the `.diagonalize()` method in SymPy to find the diagonalization of a diagonalizable matrix:

(https://docs.sympy.org/latest/tutorials/intro-tutorial/matrices.html)



Example:  Given the following matrix, is it diagonalizable?  If so, diagonalize it. 

$$
\begin{equation}
A = \left[ \begin{array}{rr} 2 & -1  \\ -1 & 2 \end{array}\right]
\end{equation}
$$

In [ ]:
# Run this cell to diagonalize A:

P, D = A.diagonalize()

P

In [ ]:
D

In [ ]:
A == P@D@P**-1

In [ ]:
# Notice, if a matrix is not diagonalizable, this function will fail

B = sp.Matrix([[2,-1],[4,-2]])
B

In [ ]:
# Uncomment and run - this will fail because B is not diagonalizable
# B.diagonalize()

# Discrete Dynamical Systems

It is often useful to describe a structure that has multiple components with a single vector.  If that structure is changing in time due to some process, it is typical to refer to the vector as a **state vector** since it describes the *state* of the structure at some particular time.  It is quite common to model such dynamic processes at discrete times and use linear transformations to model the evolution of the state vector from one time to the next.

Let's suppose that we aim to describe sequence of vectors at times $t=1, 2, 3,$... with state vectors $\mathbf{x_1}$, $\mathbf{x_2}$, $\mathbf{x_3}$.... at those times.  We propose to calculate the state vector $\mathbf{x_{k+1}}$ based only on the previous state vector $\mathbf{x_k}$.  If we model the transition from $\mathbf{x_{k+1}}$ to $\mathbf{x}_k$ with a linear transformation, then there is a matrix such that $\mathbf{x_{k+1}} = A\mathbf{x_k}$.  This sort of model is known as a **discrete dynamical system** and is used in many areas from economics to biology.

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />


## <span style='color:Red'>   Question 1  ###




We'll begin by considering two species $R$ and $S$ whose populations $k$ years after we begin recording them are $R_k$ and $S_k$.  Suppose the population change from year to year according to the rule:


$$ R_{k+1} = \frac{9}{10}R_k+ \frac{8}{10}S_k$$
$$ S_{k+1} = \frac{2}{10}R_k + \frac{9}{10}S_k$$


This is an example of a mutually beneficial relationship between two species. If species $S$ is not present, then $R_{k+1} = 0.9R_k$, which means that the population of species $R$ decreases every year. However, species $R$ benefits from the presence of species $S$ ,
 which helps $R$
 to grow by $80\%$ of the population of species $S$ .
 In the same way, $S$
 benefits from the presence of $R$ .


We will use the state vector $\mathbf{x}_k = \left[\begin{array} {c} R_k \\ S_k \end{array} \right]$ to record the population.  



<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />


## <span style='color:Red'>   Question 1a (1 pt) ###


Define the matrix $A$ such that $\mathbf{x}_{k+1} = A\mathbf{x}_{k}$

So that we don't deal with Floating Point errors, and to pass the auto-graded tests, write your answer as a SymPy matrix, with Rational Entries.  (Hint: `sp.Rational(a,b)` enters the fraction a/b as a rational number)


	     


In [ ]:
A = ...


A

In [ ]:
grader.check("q1a")

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />


## <span style='color:Red'>   Question 1b (2 pts) ###

i). Use built-in SymPy functions to find the eigenvalues of $A$.   Label them in order from largest to smallest (that is, let the largest eigenvalue be $\lambda_1$).  

ii). Use built-in SymPy functions to find a basis for each eigenspace (you do not need to write these in basis notation, for example if the basis for $\lambda_1$ is $\{\mathbf{v_1}\}$ just give $\mathbf{v_1}$ as a 2x1 SymPy matrix for your answer).  

Note: While there are an infinite number of possible basis vectors, your answers to part (ii) should have simple integer components to pass the autograder.  If they don't, go back and make sure you are writing the matrix A in part (a) using `sp.Rational(a,b)` for any decimal (otherwise you won't pass the autograder on this and future questions)




In [ ]:

...


lambda_1 = ...

lambda_2 = ...


v_1 = ...


v_2 = ...




In [ ]:
grader.check("q1b")

<!-- BEGIN QUESTION -->

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

## <span style='color:Red'>   Question 1c (2 pts) ###





i).  Do $\mathbf{v}_1$ and $\mathbf{v}_2$ form a basis for $\mathbb{R}^2$?  Justify your answer.
                
ii).  Suppose the initial populations are given by the state vector $\mathbf{x}_0 = \left[\begin{array} {c} 100 \\ 100 \end{array} \right]$.  Rewrite $\mathbf{x}_0$ as a linear combination of $\mathbf{v_1}$ and $\mathbf{v_2}$.  That is, find $c_1, c_2$ such that $\mathbf{x}_0= c_1\mathbf{v_1} + c_2\mathbf{v_2}$.




Write your answer (with justification) to **part 1ci** in this cell. 


In [ ]:


...

c1 = ...
c2 = ...




In [ ]:
grader.check("q1c")

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />


## <span style='color:Red'>   Question 1d (10 pts) ###

Answer each of the questions below in text in the same Markdown cell that follows (i.e. don't code your responses to any of these answers).   Use LaTeX for any math answers.  Note that when using dollar signs to start LaTeX don't include any spaces between your dollar signs and the math (otherwise the autograder will fail).  For example `$2x^2$` will pass, but `$_2x^2$` will fail

--------

Again, suppose $A$ is given by the matrix you found in part 1a and the initial populations are given by the state vector $\mathbf{x}_0 = \left[\begin{array} {c} 100 \\ 100 \end{array} \right]$.                                                                                                                                                                                                                                                                                                                              
 - i).  Use the linearity property of matrix multiplication along with the fact that $\mathbf{v}_1$ and $\mathbf{v}_2$ are eigenvectors of $A$ to express $\mathbf{x}_1 = A\mathbf{x}_0$  as a linear combination of $\mathbf{v}_1, \mathbf{v}_2$.  (So that you see a general pattern, do not simplify $\mathbf{x}_1$ to be a single vector, leave it expressed as a linear combination of $\mathbf{v_1}$ and $\mathbf{v_2}$ with weights.  


 - ii). In the same way, express  $\mathbf{x}_2 = A\mathbf{x}_1$ as a linear combination of $\mathbf{v}_1, \mathbf{v}_2$. If you have any exponents in your answer leave them unsimplified (for example write $5^2$ intead of $25$)


 - iii).  Finally express and $\mathbf{x}_{k+1} = A\mathbf{x}_k$ as a linear combination of $\mathbf{v}_1, \mathbf{v}_2$.   Again, leave exponents unsimplified.

 - iv).  What happens to $\mathbf{x}_k$ after a very long time? (That is, As $k$ gets very large, what are the dominant term(s) of $\mathbf{x}_{k+1}$?).    Write your answer in terms of $k$ in the form $\mathbf{x}_k\approx$ 

 - v).  When $k$ becomes very large, what happens to the ratio of the populations, $R_k/S_k$?  How is this related to the **eigenvectors** of $A$?  Explain your thinking.  


 - vi).  After a very long time (i.e. very large $k$), by approximately what factor does the population of $R$ grow each year?   What about the population of $S$?    How is this related to the **eigenvalues** of $A$? Explain your thinking. 



_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />


## <span style='color:Red'>   Question 1e (10 pts) ###



Let's look at the results from the previous part from a different perspective, using diagonalization and visualization.


In the previous question, you proved that the eigenvectors $\mathbf{v_1}$ and $\mathbf{v_2}$ form a basis of $\mathbb{R}^2$, which says that $A$ is diagonalizable.  

 - i).  Give a diagonalization of $A$.  That is, what are $P$ and $D$?   Give your answers as SymPy matrices.   (To pass the autograder, put the largest eigenvalue in the top left corner of $D$).  Hint: Use your results from the previous question.  Write any decimals as rational numbers using sp.Rational(a,b). 

 - ii).  Let $\mathcal{B}  = \{\mathbf{v_1}, \mathbf{v_2}\}$.   Suppose we start with a new initial population.  That is:
$\mathbf{x}_0 = \left[\begin{array} {c} 200 \\ 300\end{array} \right]$   Rewrite this initial population vector in terms of the basis $\mathcal{B}$. That is, what is $\{\mathbf{x_0}\}_{\mathcal{B}}$?  (Give your answer as a SymPy  2x1 Matrix). 

 - iii).  What is $\{\mathbf{x_1}\}_{\mathcal{B}}$?  (Hint, you should be able to do this with a single matrix vector multiplication).  (Give your answer as a SymPy  2x1 Matrix). 

 - iv). What is $\{\mathbf{x_2}\}_{\mathcal{B}}$?

 - v). What is $\{\mathbf{x_3}\}_{\mathcal{B}}$?

 - vi).  In general, what is $\{\mathbf{x_{k}}\}_{\mathcal{B}}$?  Write your answer in the Markdown cell below as a single vector, in terms of $k$.

 - vii).  How can we use matrix-vector multiplication to convert $\{\mathbf{x_{k}}\}_{\mathcal{B}}$ back into our standard basis?    That is, given $\{\mathbf{x_{k}}\}_{\mathcal{B}}$ from the previous part, what is $\mathbf{x_k}$?  Write your answer in the Markdown cell below, first as a matrix-vector product and then simplified to a single vector. 

 - viii). When $k$ becomes very large, what happens to the ratio of the populations, $R_k/S_k$?


**Give your written answers to parts vi, vii and viii in this cell:  Use LaTeX to write any math.**

In [ ]:

# Answers to part (i): 


P = ...

D = ...

#check your diagonalization - the following should return True if you have the correct diagonalization

print(A == P@D@P**-1)


# Answer to part(ii) Give your answer for x_B as a SymPy 2x1 Matrix:
x_B = ...

# Answer to part(iii) Give your answer for x_B as a SymPy 2x1 Matrix:

x_1_B = ...

# Answer to part(iv) Give your answer for x_B as a SymPy 2x1 Matrix:

x_2_B = ...

# Answer to part(v) Give your answer for x_B as a SymPy 2x1 Matrix:

x_3_B = ...



In [ ]:
grader.check("q1e")

<!-- END QUESTION -->

Let's visualize this problem.  Note in the graphs below, the scale of the axes is in hundreds

<img src="img/phase_plane.png" width="600" height="300">



<img src="img/phase_plane_2.png" width="600" height="300">

This problem demonstrates the power of using eigenvalues and eigenvectors to rewrite the discrete dynamical system in terms of a new coordinate system. By doing so, we are able to predict the long-term behavior of the populations independently of the initial populations.
Diagrams like those shown in Figure 4.4.1 are called phase portraits.


# Markov Chains

This section continues this exploration by looking at Markov chains, which form a specific type of discrete dynamical system. 

### Some Terminology

**Definitions**:   
A vector whose entries are nonnegative and add to 1 is called a **probability vector**. 

A square matrix whose columns are probability vectors is called a **stochastic matrix**.

If $A$ is a stochastic matrix, we say that a probability vector $\mathbf{q}$ 
 is a **steady-state** or **stationary vector** if $A\mathbf{q} = \mathbf{q}$.    



<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />


## <span style='color:Red'>   Question 2  ###


Consider the following two **stochastic** matrices:
\begin{equation*}
C=\begin{bmatrix}0 & 1\\ 1 & 0\end{bmatrix}
\qquad
F=\begin{bmatrix}0.4 & 0.3\\ 0.6 & 0.7\end{bmatrix}
\end{equation*}


<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />


## <span style='color:Red'>   Question 2a (2 pts) ###

i).  Find the eigenvalues of $C$.  List the largest one first (i.e. the largest is $\lambda_1$)

ii).  Then find a steady-state vector, $\mathbf{q}$ for $C$.  Give your answer as a **probability vector.**  Write as a SymPy 2x1 matrix.
                                    



In [ ]:
...


eig_1 = ...
eig_2 = ...

q = ...


In [ ]:
grader.check("q2a")

<!-- BEGIN QUESTION -->

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />


## <span style='color:Red'>   Question 2b (5 pts) ###


Given a stochastic matrix $A$ and a probability vector $\mathbf{x}_0$, we can form the sequence $\mathbf{x}_k$, where $\mathbf{x}_{k+1} = A\mathbf{x}_k$.  We call this sequence of vectors a **Markov chain**.  It turns out that we can guarantee the vectors $\mathbf{x_k}$ are also probability vectors (see Section 4.5.2 of Understanding Linear Algebra 
https://understandinglinearalgebra.org/sec-stochastic.html#sec-stochastic for a proof). 


Let's examine the Markov chain that begins with the vector $\mathbf{x}_0 = \left[\begin{array} {c} 1 \\ 0 \end{array} \right]$


 - i). Complete the function below to take a stochastic matrix $A$ and a probability vector $\mathbf{x}_0$ as input and construct the first $n$ terms of the Markov chain.  The first term in the list should be $\mathbf{x}_0$.  Write this function so it works function regardless if you give it a SymPy matrix or a NumPy array. 
Save this chain in a Python list.

 - ii).  Then run the function with the matrix $C$ and the vector $\mathbf{x}_0$ as input for $n=10$

 - iii). What do you notice about this Markov chain?  Does it converge to the steady state vector for $C$?   


Write your answer to 2b iii in this cell

In [ ]:

def markov_chain(A, x0, n):
    ...

In [ ]:
# Then run markov_chain using C, x0 and n=10:

...

# Run markov_chain using C, x0 and n=10, and save the list to the variale markov_chain1
markov_chain1 = ...

markov_chain1


In [ ]:
grader.check("q2b")

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />


## <span style='color:Red'>   Question 2c (5 pts) ###

Now consider $F=\begin{bmatrix}0.4 & 0.3\\ 0.6 & 0.7\end{bmatrix}$


 - i).  Find the eigenvalues of $F$.  List the largest one first (i.e. the largest is $\lambda_1$).  

 - ii).  Then find a steady-state vector, $\mathbf{q}$ for $F$.  Give your answer as a **probability vector.**  Write any fractions as fractions (do not round them).  Enter as a SymPy 2x1 matrix.

 - iii). As before, find the first 10 terms of the Markov chain beginning with $\mathbf{x}_0 = \left[\begin{array} {c} 1 \\ 0 \end{array} \right]$ and $\mathbf{x}_{k+1} = F\mathbf{x_k}$

 - iv).  What do you notice about this Markov chain? Does it converge to the steady-state vector for $F$?

 - v).  What if you change the initial starting probability vector?  Find the first 10 terms of the Markov chain beginning with $\mathbf{x}_{0new} = \left[\begin{array} {c} .6 \\ .4 \end{array} \right]$ and $\mathbf{x}_{k+1} = F\mathbf{x_k}$.   Does it converge to the steady-state vector for $F$?
                                                                                     

                                    



Write your answers to **2c iv** and **2cv** in this cell

In [ ]:
...


eig_1_F = ...
eig_2_F = ...

q_F = ...

# Run markov_chain using F, x0 and n=10, and save the list to the variable markov_chain2
markov_chain2 = ...

# Run markov_chain using F, x0_new and n=10, and save the list to the variable markov_chain3
markov_chain3 = ...


print(markov_chain2)

markov_chain3



In [ ]:
grader.check("q2c")

<!-- END QUESTION -->

Here are a few important facts about the eigenvalues of **any stochastic** matrix.

1).  $\lambda = 1$ is an eigenvalue of **any** stochastic matrix (see Section 4.5 in Understanding Linear Algebra for a proof; 
https://understandinglinearalgebra.org/sec-stochastic.html#exercise-stochastic-eigenvalue) 

We usually order the eigenvalues so it is the first eigenvalue meaning $\lambda_1 = 1$

2).  Any stochastic matrix has at least one steady state vector $\mathbf{q}$.

3). All other eigenvalues satisfy the property that $|\lambda_j| \leq 1$

4). If all but the first eigenvalue satisfy $|\lambda_j|<1$, then there is a unique steady-state vector $\mathbf{q}$ and any Markov chain will converge to $\mathbf{q}$.  This was the case for the matrix $F$ in the previous question.
 




## Navigating Webpages:  Google PageRank

When I recently googled the phrase “linear algebra,” Google told me there were about
179 million results and that the Wikipedia page was the one most likely to be useful to
me. How does Google decide to recommend that page above all the others? Well, Google
computes a quantity called PageRank for each page on the Internet. Pages with a higher
PageRank are deemed to be more valuable. The key to computing PageRank lies in an
algorithm that we’ll study in this lab.

    
One could determine the importance of a web page by allowing people to vote for
their favorite web pages. However, Google wants their rankings to be free of any human
bias so it examines the structure of the Internet itself to rank pages. Here’s the idea: if
my home page is valuable, many other pages will link to it. Of course, I could game
the rankings by creating lots of pages that link to my home page. So Google views a
page as being more valuable when valuable pages link to it. For instance, if three of my
friends link to my home page, that means three people think I have a valuable page. But if
nytimes.com, amazon.com, and beyonce.com link to me, that probably means more
people will find my page to be valuable so I should have a higher PageRank.

                        
Here’s how to compute PageRank:
 - Each webpage has a certain amount of PageRank
that we denote by $x_i$. 
 - Each page divides its PageRank into equal portions, one for each
outgoing link. 
 - Each page then gives one portion of its PageRank to each page it links to.
 - A page’s PageRank is then the sum of all the PageRank it receives from pages that link to
it. 
                        
                        
Let’s look at an example.




Suppose that our Internet only has three pages with links as shown below. 

<img src="img/internet_1.png" width="200" height="200">

                        
Notice:
                        
 - Page 2 only links to page 1 so it gives all of its PageRank $x_2$ to Page 1. 

 - Page 3 has two outgoing links so it divides its PageRank $x_3$ into two equal pieces and gives half its
PageRank $\frac{1}{2}x_3$ to Page 1. 

The PageRank $x_1$ is therefore: $$x_1 = x_2+\frac{1}{2}x_3$$

<!-- BEGIN QUESTION -->

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />


## <span style='color:Red'>   Question 3a (2 pts) ###

Find similar expressions for $x_2$ and $x_3$.

Write your answer below using LaTeX



_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />


## <span style='color:Red'>   Question 3b (2 pts) ###

We now form the PageRank vector $\mathbf{x} = \left[\begin{array} {c} x_1 \\ x_2 \\ x_3 \end{array} \right]$.  


 - i).  Find a matrix $G$ such that the expressions for $x_1, x_2, x_3$ you found above can be written in the form $G\mathbf{x}$.  (You do not need to express rationals using sp.Rational for this part).

 - ii).  Explain why $G$ a stochastic matrix.  Why will this always be the case for any internet, assuming that each page has outgoing links?

The matrix $G$ is called the "Google matrix."


Write your answer as a SymPy Matrix. Do NOT use rational to write any decimals. 

Write your answer to **3b ii**  in this cell

In [ ]:

G = ...

G

In [ ]:
grader.check("q3b")

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />


## <span style='color:Red'>   Question 3c (5 pts) ###


 - i).  Find the eigenvalues of $G$.  List the largest one first (i.e. the largest is $\lambda_1$).   


 - ii).  Then find a steady-state vector, $\mathbf{q}$ for $G$.  Give your answer as a **probability vector.**  Write as a SymPy 2x1 matrix.  Again, do not round any entries of $\mathbf{q}$.



- iii).  Which of the three webpages has the highest PageRank and which has the smallest?   (The webpages are labeled 1, 2 and 3 in the diagram above).  `



- iv). Starting with the initial state vector $\mathbf{x_0} = \left[\begin{array} {c} 1\\0 \\ 0 \end{array} \right]$ use the function `markov_chain` you wrote above to generate a Markov chain with $20$ vectors.   


 - v). Based on the eigenvalues of $G$, what do you know about the long-term behavior of **any** Markov chain constructed using $G$? (i.e. will it converge and if so, to what?)


                                                                                     

                                    



Write your answer to **3c v**  in this cell

In [ ]:
# Answers to 3c (i) - (iv) below:

...

eig_1_G = ...
eig_2_G = ...


q_G = ...

highest_PageRank = ...

smallest_PageRank = ...

# Run markov_chain using G, x0 and n=20, and save the list to the variable markov_chain_G
markov_chain_G = ...



markov_chain_G




In [ ]:
grader.check("q3c")

<!-- END QUESTION -->




The problem above show us two ways to find the PageRank vector. In the first, we determine a steady-state vector directly by finding a description of the eigenspace 
$E_1$ and then finding the appropriate scalar multiple of a basis vector that gives us the steady-state vector. To find a description of the eigenspace ,$E_1$
 however, we need to find the null space $Nul(G-I)$.
 

Remember that the real Internet has 35 trillion pages so finding $Nul(G-I)$
 requires us to row reduce a matrix with 35 trillion rows and columns. 
As we saw in Lab 1, that is not computationally feasible. 

    
As suggested by the activity, the second way to find the PageRank vector is to use a Markov chain that converges to the PageRank vector. Since multiplying a vector by a matrix is significantly less work than row reducing the matrix, this approach is computationally feasible, and it is, in fact, how Google computes the PageRank vector.



<!-- BEGIN QUESTION -->

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />


## <span style='color:Red'>   Question 3d (4 pts) ###

Now consider the Internet with eight web pages, shown here:

<img src="img/internet_2.png" width="400" height="400">



 - i).  Construct the Google matrix $G_8$ for this Internet.  Write your answer as a Numpy array.

 - ii).  Use a Markov chain to find the steady-state Page Rank vector $\mathbf{q}$ for $G_8$.  Give the answer as a numpy 8x1 array, with each value rounded to the nearest thousandth (make sure you run your Markov chain long enough that each component has converged to the nearest thousandth). 

 - iii).   Which of the  webpages has the highest PageRank and which has the smallest?   


                                                                      


In [ ]:




G8 = np.array([
[0, 1/2, 1/2, 0,  0,  0,  0,  0],
[0, 0, 0, 1,  0,  0,  0,  0],
[0, 1/2, 0, 0, 1/2, 0, 0, 0],
[0, 1/2, 0, 0, 0, 1/2, 0, 0],
[0, 0, 0, 0, 0, 1/3, 1/3, 1/3],
[0, 0, 0, 0, 0, 0, 0, 1],
[1/3, 0, 0, 0, 1/3, 0, 0, 1/3],
...


...


q_G8 = ...

highest_PageRank_G8 = ...

smallest_PageRank_G8 = ...

print(q_G8)





In [ ]:
grader.check("q3d")

<!-- END QUESTION -->

These were two simplified versions of the Internet.  See Section 4.5 of Understanding Linear Algebra for how Google handles more complicated arrangements:
                                                                                                    
https://understandinglinearalgebra.org/sec-stochastic.html#subsec-google

<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />


### Submission Instructions

Before proceeding any further, **save this notebook.**

Then run the cell below to double check that you don't have any spaces between dollar signs and text when writing LaTeX:

In [ ]:
# Run this cell before you run the 'grader.export()' cell below.  
# It will search for LaTeX errors that will cause the LaTeX compiler to fail.  

import simple_latex_checker as slc

nb = slc.Nb_checker()
nb.run_check("lab04.ipynb")

After running the `grader.export()` cell provided below, **2 files will be created**: a zip file and pdf file.  You can download them using the links provided below OR by finding them in the same folder where this juptyer notebook resides in your JuptyerHub.

To receive credit on this assignment, **you must submit BOTH of these files
to their respective Gradescope portals:** 


* **Lab 4 Autograded**: Submit the zip file that is output by the `grader.export()` cell below to the Lab 4 Autograded assignment in Gradescope.

* **Lab 4 Manually Graded**: Submit your lab04.PDF to the Lab 4 Manually Graded assignment in Gradescope.  **It is your responsibility to fully review your PDF file before submitting and make sure that all your lines of code are visible and any LaTeX has correctly compiled and is fully viewable.**  **YOU MUST SELECT THE PAGES CORRESPONDING TO EACH QUESTION WHEN YOU UPLOAD TO GRADESCOPE.** If not, you will lose points.    

[TROUBLESHOOTING TIPS](https://docs.google.com/document/d/1ndr3Wj1PSF5qzlLMaBJznwh6QGeEXjd5TAJ6nf9EJvo/edit?usp=sharing)  If you are having any issues compiling your assignment, please read through these troubleshooting tips first, then post any questions on Piazza.  

**You are responsible for ensuring your submission follows our requirements. We will not be granting regrade requests nor extensions to submissions that don't follow instructions.** If you encounter any difficulties with submission, please don't hesitate to reach out to staff prior to the deadline.

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

AFTER running the cell below, click on <a href='lab04.pdf' download>this link to download the PDF </a> to upload to Gradescope.  There will be a separate link that appears after running the cell below with a link to download the zip file to upload to Gradescope.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(run_tests=True)